In [3]:
import gdown

file_id = "1ToQBXu2QU17WYcCQQ44YKxFPSf1kvYRr"  # ← 여기에 본인의 ID 넣기
output_name = "VOCdevkit.zip"    # 저장할 파일명

# 다운로드 실행
gdown.download(f"https://drive.google.com/uc?id={file_id}", output_name, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1ToQBXu2QU17WYcCQQ44YKxFPSf1kvYRr
To: /content/VOCdevkit.zip
100%|██████████| 4.54M/4.54M [00:00<00:00, 30.6MB/s]


'VOCdevkit.zip'

In [5]:
import zipfile
import os

zip_path = "VOCdevkit.zip"           # 압축 파일 경로
extract_path = "VOCdevkit" # 압축 해제 대상 경로

# 압축 해제
if not os.path.exists(extract_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall()
    print(f"✅ 압축 해제 완료: {extract_path}")
else:
    print("📂 이미 압축이 풀려 있습니다.")


📂 이미 압축이 풀려 있습니다.


#### JPEGImages 디렉토리에 있는 임의의 이미지 보기

In [ ]:
import cv2
import matplotlib.pyplot as plt
import os

default_dir = '.'
img = cv2.imread(os.path.join(default_dir, 'VOCdevkit/VOC2007/JPEGImages/000032.jpg'))
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
print('img shape:', img.shape)

plt.figure(figsize=(8, 8))
plt.imshow(img_rgb)
plt.show()


#### Annotations 디렉토리에 있는 임의의 annotation 파일 보기

In [ ]:
!cat VOCdevkit/VOC2007/Annotations/000032.xml

#### 임의의 maksing 이미지 보기

In [ ]:

# img = cv2.imread(os.path.join(default_dir, 'VOCdevkit/VOC2007/SegmentationClass/000032.png'))
img = cv2.imread(os.path.join(default_dir, 'VOCdevkit/VOC2007/SegmentationObject/000032.png'))

img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
print('img shape:', img.shape)

plt.figure(figsize=(8, 8))
plt.imshow(img_rgb)
plt.show()

#### Annotation xml 파일에 있는 요소들을 파싱하여 접근하기
* ElementTree를 이용하여 XML 파싱

In [ ]:
import os
import random

VOC_ROOT_DIR ="VOCdevkit/VOC2007/"
ANNO_DIR = os.path.join(VOC_ROOT_DIR, "Annotations")
IMAGE_DIR = os.path.join(VOC_ROOT_DIR, "JPEGImages")

xml_files = os.listdir(ANNO_DIR)
print(xml_files[:5]); print(len(xml_files))

In [ ]:
# !pip install lxml
import os
import xml.etree.ElementTree as ET

xml_file = os.path.join(ANNO_DIR, '000032.xml')

# XML 파일을 Parsing 하여 Element 생성
tree = ET.parse(xml_file)
root = tree.getroot()

# image 관련 정보는 root의 자식으로 존재
image_name = root.find('filename').text
full_image_name = os.path.join(IMAGE_DIR, image_name)
image_size = root.find('size')
image_width = int(image_size.find('width').text)
image_height = int(image_size.find('height').text)

# 파일내에 있는 모든 object Element를 찾음.
objects_list = []
for obj in root.findall('object'):
    # object element의 자식 element에서 bndbox를 찾음.
    xmlbox = obj.find('bndbox')
    # bndbox element의 자식 element에서 xmin,ymin,xmax,ymax를 찾고 이의 값(text)를 추출
    x1 = int(xmlbox.find('xmin').text)
    y1 = int(xmlbox.find('ymin').text)
    x2 = int(xmlbox.find('xmax').text)
    y2 = int(xmlbox.find('ymax').text)

    bndbox_pos = (x1, y1, x2, y2)
    class_name=obj.find('name').text
    object_dict={'class_name': class_name, 'bndbox_pos':bndbox_pos}
    objects_list.append(object_dict)

print('full_image_name:', full_image_name,'\n', 'image_size:', (image_width, image_height))

for object in objects_list:
    print(object)



#### Annotation내의 Object들의 bounding box 정보를 이용하여 Bounding box 시각화

In [ ]:
import cv2
import os
import xml.etree.ElementTree as ET

xml_file = os.path.join(ANNO_DIR, '000032.xml')

tree = ET.parse(xml_file)
root = tree.getroot()

image_name = root.find('filename').text
full_image_name = os.path.join(IMAGE_DIR, image_name)

img = cv2.imread(full_image_name)
# opencv의 rectangle()는 인자로 들어온 이미지 배열에 그대로 사각형을 그려주므로 별도의 이미지 배열에 그림 작업 수행.
draw_img = img.copy()
# OpenCV는 RGB가 아니라 BGR이므로 빨간색은 (0, 0, 255)
green_color=(0, 255, 0)
red_color=(0, 0, 255)

# 파일내에 있는 모든 object Element를 찾음.
objects_list = []
for obj in root.findall('object'):
    xmlbox = obj.find('bndbox')

    left = int(xmlbox.find('xmin').text)
    top = int(xmlbox.find('ymin').text)
    right = int(xmlbox.find('xmax').text)
    bottom = int(xmlbox.find('ymax').text)

    class_name=obj.find('name').text

    # draw_img 배열의 좌상단 우하단 좌표에 녹색으로 box 표시
    cv2.rectangle(draw_img, (left, top), (right, bottom), color=green_color, thickness=1)
    # draw_img 배열의 좌상단 좌표에 빨간색으로 클래스명 표시
    cv2.putText(draw_img, class_name, (left, top - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, red_color, thickness=1)

img_rgb = cv2.cvtColor(draw_img, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(10, 10))
plt.imshow(img_rgb)
plt.show()